# Python Analytics Project: Analysis of student performance

## Problem

We have data of student performance, we are required to analyze completed lessons and answer the following questions:

__1__. How many students have successfully finished only one course? (Successful completion is when a student pass the course exam)

__2__. Which exam is the most difficult and simplest: which courses and exams within the courses have the lowest and highest completion rate? (_completion rate - number of successful exams / number of all attempts to pass the exam_)

__3__. What is the average period of passing exams in each course? (by passing we mean the last successful passing of the exam by a student)

__4__. Which courses are the most popular by the number of registrations for them? Which courses have the largest outflow?

__5__. In which semester was the lowest completion of courses and the longest average terms of completion of courses observed? (In the period from the beginning of 2013 to the end of 2014)

## Data

There are four tables, and their names and column definitions are listed below:

### Table 1: assessments

_This table includes information about the scores in the test. Usually, each subject in a semester includes a series of tests with grades, followed by a final examination test (exam)_

__code_module__ - subject (identification code);

__code_presentation__ — semester (identification code);

__id_assessment__ — test (assessment identification code);

__assessment_type__ — type of test. There are three types of assessment: teacher assessment (TMA), computer assessment (SMA), course exam (Exam);

__date__ — information about the final date of the test. Calculated as the number of days since the beginning of the semester. The start date of the semester has the number 0;

__weight__ — the weight of the test in % in the assessment for the course. Usually exams are considered separately and have a weight of 100%; the sum of all other grades is 100%.

### Table 2: courses

_This table contains a list of subjects by semester_

__code_module__ — subject (identification code);

__code_presentation__ — semester (identification code);

__module_presentation_length__ — duration of the semester in days.

### Table 3: studentAssessment

_This table contains the results of the students' tests. If the student does not send the work for evaluation, the result is not recorded in the table_

__id_assessment__ — test (assessment identification code);

__id_student__ — student identification number;

__date_submitted__ — the date of the student's test, measured as the number of days since the beginning of the semester;

__is_banked__ — the fact that the test result was transferred from the last semester (when the student returned from academic leave);

__score__ — the student's score in this test. The range is from 0 to 100. Score below 40 failed test.

### Table 4: studentRegistration

_This table contains information about the time when the student registered for the course in the semester_

__code_module__ — subject (identification code);

__code_presentation__ — semester (identification code);

__id_student__ — student identification number;

__date_registration__ — date of registration of the student. This is the number of days measured from the beginning of the semester (for example, a negative value of -30 means that the student has registered for the course 30 days before it starts);

__date_unregistration__ — the date of cancellation of the student's registration from the subject. For students who have completed the course, this field remains empty.

In [177]:
# importing libraries
import pandas as pd

In [178]:
# reading tables and saving to dataframe
assessments = pd.read_csv('assessments.csv')
courses = pd.read_csv('courses.csv')
std_asmt = pd.read_csv('studentAssessment.csv')
std_reg = pd.read_csv('studentRegistration.csv')

Let's take a look at the appearence of these tables

In [179]:
assessments.head(3)

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.00%,10.00%
1,AAA,2013J,1753,TMA,54.00%,20.00%
2,AAA,2013J,1754,TMA,117.00%,20.00%


In [180]:
courses.head(3)

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268


In [181]:
std_asmt.head(3)

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.00%
1,1752,28400,22,0,70.00%
2,1752,31604,17,0,72.00%


In [182]:
std_reg.head(3)

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.00%,NaN
1,AAA,2013J,28400,-53.00%,NaN
2,AAA,2013J,30268,-92.00%,12.00%


First of all check the data in every table

In [183]:
# check for empty values
assessments.isna().sum()

code_module           0
code_presentation     0
id_assessment         0
assessment_type       0
date                 11
weight                0
dtype: int64

There are empty values in the column with dates. It's ok, because this field is filled in only for tests and it's empty for exams

In [184]:
# check duplicates
assessments.loc[assessments.duplicated()]

,code_module,code_presentation,id_assessment,assessment_type,date,weight


There are no duplicated values in "assessments" table

In [185]:
# check for empty values
courses.isna().sum()

code_module                   0
code_presentation             0
module_presentation_length    0
dtype: int64

In [186]:
# check duplicates
courses.loc[courses.duplicated()]

,code_module,code_presentation,module_presentation_length


There are no empty and duplicated values in "courses" table

In [187]:
# check for empty values
std_asmt.isna().sum()

id_assessment       0
id_student          0
date_submitted      0
is_banked           0
score             173
dtype: int64

In [188]:
# filter table by NaN in the score
std_asmt[std_asmt.isna().any(axis=1)].head(3)

,id_assessment,id_student,date_submitted,is_banked,score
215,1752,721259,22,0,NaN
937,1754,260355,127,0,NaN
2364,1760,2606802,180,0,NaN


We can see a lot of empty values in the score column, it looks strange, because we have the column "date_submitted" filled in. It seems that it's a mistake in data, may be a student passed the test but a teacher hasn't given a grade

We should delete rows with NaN score 

In [189]:
# deleting rows with NaN score
std_asmt = std_asmt.drop(std_asmt.index[std_asmt[std_asmt.isna().any(axis=1)].index])

In [190]:
# check duplicates
std_asmt.loc[std_asmt.duplicated()]

,id_assessment,id_student,date_submitted,is_banked,score


There are no duplicated values in "std_asmt" table

In [191]:
# check for empty values
std_reg.isna().sum()

code_module                0
code_presentation          0
id_student                 0
date_registration         45
date_unregistration    22521
dtype: int64

There are empty values in the "date_registration"and "date_unregistration" columns, it looks like the truth

But if we have empty values in both of them simultaneously, it has no sense and we should delete these rows

In [192]:
# filter table by nan in both columns and determine rows with nan dates
std_reg_na = std_reg[std_reg['date_registration'].isna()]
std_reg_na = std_reg_na[std_reg_na['date_unregistration'].isna()].index
std_reg_na

Int64Index([2344, 12893, 14392, 14393, 17559, 23796], dtype='int64')

In [193]:
# deleting rows with NaN 
std_reg = std_reg.drop(std_reg.index[[2344, 12893, 14392, 14393, 17559, 23796]])

In [194]:
# check duplicates
std_reg.loc[std_reg.duplicated()]

,code_module,code_presentation,id_student,date_registration,date_unregistration


There are no duplicated values in "std_reg" table

### 1st question

We need to find the number of students who have successfully completed the course.
Successful completion of the course is a successful passing of exams on the subject in both semesters

In [195]:
# join tables std_asmt and assessments
merge_assesments = std_asmt.merge(assessments, on = 'id_assessment')
merge_assesments.head(3)

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
0,1752,11391,18,0,78.00%,AAA,2013J,TMA,19.00%,10.00%
1,1752,28400,22,0,70.00%,AAA,2013J,TMA,19.00%,10.00%
2,1752,31604,17,0,72.00%,AAA,2013J,TMA,19.00%,10.00%


We will select only those results where the number of points > = 40 (successful passing) and type of assessment is exam. Also will aggregate data by the number of courses and leave only those who have passed just one course

In [196]:
merge_assesments \
    .query("score >= 40 and assessment_type == 'Exam'") \
    .groupby('id_student', as_index = False) \
    .agg({'code_module':'count'}) \
    .query('code_module == 1') \
    .rename(columns={'code_module':'quantity_exm'}) \
    .count()

id_student      3802
quantity_exm    3802
dtype: int64

__The answer:__ 3802 students successfully completed just one course

### 2nd question

We are required to find the most difficult and simplest exam: which subject and exams within the subject have the lowest and highest completion rate 
(completion rate: number of successful exams / number of all attempts to pass the exam)

In [197]:
# creating a table with the total number of attempts to pass exams
all_attempts = merge_assesments \
    .query("assessment_type == 'Exam'") \
    .groupby('id_assessment', as_index = False) \
    .agg({'id_student':'count'}) \
    .rename(columns={'id_student':'all_attempts'})
all_attempts

,id_assessment,all_attempts
0,24290,747
1,24299,1168
2,25340,602
3,25354,968
4,25361,524
5,25368,950


In [198]:
# creating a table with the number of successfully passed exams
success_attempts = merge_assesments \
    .query("score >= 40 and assessment_type == 'Exam'") \
    .groupby(['id_assessment', 'code_module'], as_index = False) \
    .agg({'id_student':'count'}) \
    .rename(columns={'id_student':'success_attempts'})
success_attempts

,id_assessment,code_module,success_attempts
0,24290,CCC,664
1,24299,CCC,1019
2,25340,DDD,504
3,25354,DDD,878
4,25361,DDD,485
5,25368,DDD,842


In [199]:
# joining obtained tables
merge_attempts = success_attempts.merge(all_attempts, on='id_assessment')

In [200]:
# calculation of finalizability
merge_attempts['finalizability'] = round(merge_attempts.success_attempts / merge_attempts.all_attempts * 100, 2)

In [201]:
# sorting from larger to smaller and changing format to percentage
merge_attempts = merge_attempts.sort_values('finalizability', ascending = False)
pd.set_option('display.float_format', '{:.2f}%'.format)
merge_attempts

,id_assessment,code_module,success_attempts,all_attempts,finalizability
4,25361,DDD,485,524,92.56%
3,25354,DDD,878,968,90.70%
0,24290,CCC,664,747,88.89%
5,25368,DDD,842,950,88.63%
1,24299,CCC,1019,1168,87.24%
2,25340,DDD,504,602,83.72%


In [202]:
# calculation of average finalizability by subjects
merge_attempts \
    .groupby('code_module', as_index = False) \
    .agg({'finalizability':'mean'}) \
    .sort_values('finalizability', ascending = False)

,code_module,finalizability
1,DDD,88.90%
0,CCC,88.06%


__The answer:__ exam 25361 has the highest finalizability among students; exam 25340 has the lowest.
Subject DDD has higher finalizability than CCC

### 3rd question

We need to find the average period of passing exams in each subject
(by passing we mean the last successful passing of the exam by a student)

In [203]:
# removing percentage format for convenient calculation
pd.set_option('display.float_format', '{:.2f}'.format)

In [204]:
# determination of the average period of passing exams in subjects
merge_assesments \
    .query("score >= 40 and assessment_type == 'Exam'") \
    .groupby('code_module', as_index = False) \
    .agg({'date_submitted':'mean'}) \
    .rename(columns={'date_submitted':'mean_time'}) \
    .round(2)

,code_module,mean_time
0,CCC,239.35
1,DDD,237.98


__The answer:__ The average period of passing exam in subject CCC is 239.35 days and 237.98 days for DDD subject

### 4th question

We are required to find the most popular courses by the number of registrations and courses with the largest outflow

In [205]:
# determing courses with the most popular courses by the number of registrations
std_reg \
    .groupby('code_module', as_index = False) \
    .id_student.nunique() \
    .rename(columns={'id_student':'st_quantity'}) \
    .sort_values('st_quantity', ascending = False) \
    .head(3)

,code_module,st_quantity
1,BBB,7691
5,FFF,7396
3,DDD,5846


In [206]:
# determing course with the largest outflow
std_reg \
    .query('date_unregistration > 0') \
    .groupby('code_module', as_index = False) \
    .id_student.nunique() \
    .rename(columns={'id_student':'st_quantity'}) \
    .sort_values('st_quantity', ascending = False) \
    .head(3)

,code_module,st_quantity
5,FFF,1599
3,DDD,1531
2,CCC,1387


__The answer:__ BBB is the most most popular courses by the number of registrations, FFF course has the largest outflow

### 5th question

We need to find semester with the lowest completion of courses and the longest average terms of completion of courses (period from the beginning of 2013 to the end of 2014)

In [207]:
# creating a function that allows you to build a cohort (semester) analysis
def generate_cohorts(df):
# selection the number of all students' attempts to pass the exam
    all_attempts = df \
        .query("assessment_type == 'Exam'") \
        .groupby('code_presentation', as_index = False) \
        .agg({'id_student':'count'}) \
        .rename(columns={'id_student':'all_attempts'})
# successful attempts
    success_attempts = df \
        .query("score >= 40 and assessment_type == 'Exam'") \
        .groupby('code_presentation', as_index = False) \
        .agg({'id_student':'count', 'date_submitted':'mean'}) \
        .rename(columns={'id_student':'success_attempts', 'date_submitted':'mean_time'})
# joining tables
    cohorts = success_attempts.merge(all_attempts, on='code_presentation')
# calculation of average completion rate of exams
    cohorts['finalizability'] = round(cohorts.success_attempts / cohorts.all_attempts * 100, 2)
    cohorts = cohorts.drop(columns=['success_attempts', 'all_attempts'])
    return cohorts

In [208]:
generate_cohorts(merge_assesments)

,code_presentation,mean_time,finalizability
0,2013B,230.16,83.72
1,2013J,239.51,90.70
2,2014B,233.00,90.40
3,2014J,243.68,87.87


__The answer:__ The lowest completion rate was in semester 2013B, and the longest average completion terms were in semester 2014J

### RFM analysis

For qualitative analysis of the audience, we will use a segmentation-based approach. We will build adapted clusters of students to evaluate the audience qualitatively using RFM analysis. 

Metrics: R is the average time of passing one exam, F is the completion of courses, M is the average number of points received for the exam.

In [209]:
# creating a function for calculating indicators
# counting the averages from the full table to take into account those who did not pass the exam
def generate_new_cohorts(df):
# calculating the average score and the average time of passing the exam among all attempts
    all_attempts = df \
        .query("assessment_type == 'Exam'") \
        .groupby('id_student', as_index = False) \
        .agg({'id_assessment':'count', 'score':'mean', 'date_submitted':'mean'}) \
        .rename(columns={'id_assessment':'all_attempts', 'date_submitted':'mean_time', 'score':'mean_score'})
# calculating the total number of successful exams
    success_attempts = df \
        .query("score >= 40 and assessment_type == 'Exam'") \
        .groupby('id_student', as_index = False) \
        .agg({'id_assessment':'count'}) \
        .rename(columns={'id_assessment':'success_attempts'})
# joining results
    cohorts = success_attempts.merge(all_attempts, on='id_student', how = 'right')
# calculation of average completion rate of exams
    cohorts['finalizability'] = round(cohorts.success_attempts / cohorts.all_attempts * 100, 2)
    cohorts = cohorts.drop(columns=['success_attempts', 'all_attempts']).fillna(0)
    return cohorts

In [218]:
# received a table with indicators for each student
rfm = generate_new_cohorts(merge_assesments)
rfm

,id_student,mean_score,mean_time,finalizability
0,23698,80.00,243.00,100.00
1,24213,58.00,236.00,100.00
2,27116,96.00,243.00,100.00
3,28046,40.00,237.00,100.00
4,28787,44.00,243.00,100.00
...,...,...,...,...
4628,2694886,69.00,236.00,100.00
4629,2694933,73.00,230.00,100.00
4630,2695608,73.00,237.00,100.00
4631,2697181,80.00,230.00,100.00


Now we need to choose the boundaries of metrics to create clusters

In [219]:
#use quantiles to evenly distribute the metrics into three groups
quintiles = rfm[['mean_time', 'mean_score']].quantile([.33, .66]).to_dict()
quintiles

{'mean_time': {0.33: 236.0, 0.66: 243.0},
 'mean_score': {0.33: 56.0, 0.66: 76.0}}

Since there are only three values in the finalizability field, we use only them: 0, 50, 100

__RFM__

__R__ - mean_time

__F__ - finalizability

__M__ - mean_score

In [220]:
# creating a function for "frequency" (finalizability), where 1 is assigned to max values
def f_score(x):
    if x == 0:
        return 3
    elif x == 50:
        return 2
    else:
        return 1

In [221]:
# creating a function for "recency" and "monetary" (mean_time и mean_score), where 1 is assigned to max values
def rm_score(x, c):
    if x <= quintiles[c][.33]:
        return 3
    elif x <= quintiles[c][.66]:
        return 2
    else:
        return 1  

In [222]:
# adding cluster numbers to the table
rfm['R'] = rfm['mean_time'].apply(lambda x: rm_score(x, 'mean_time'))
rfm['F'] = rfm['finalizability'].apply(lambda x: f_score(x))
rfm['M'] = rfm['mean_score'].apply(lambda x: rm_score(x, 'mean_score'))

In [223]:
# adding new column with RFM score
rfm['RFM Score'] = rfm['R'].map(str) + rfm['F'].map(str) + rfm['M'].map(str)
rfm

,id_student,mean_score,mean_time,finalizability,R,F,M,RFM Score
0,23698,80.00,243.00,100.00,2,1,1,211
1,24213,58.00,236.00,100.00,3,1,2,312
2,27116,96.00,243.00,100.00,2,1,1,211
3,28046,40.00,237.00,100.00,2,1,3,213
4,28787,44.00,243.00,100.00,2,1,3,213
...,...,...,...,...,...,...,...,...
4628,2694886,69.00,236.00,100.00,3,1,2,312
4629,2694933,73.00,230.00,100.00,3,1,2,312
4630,2695608,73.00,237.00,100.00,2,1,2,212
4631,2697181,80.00,230.00,100.00,3,1,1,311


In [227]:
# group by the resulting clusters and counting the average
rfm \
    .groupby('RFM Score', as_index = False) \
    .agg({'mean_score':'mean', 'mean_time':'mean', 'finalizability':'mean', 'id_student' : 'count'}) \
    .rename(columns={'id_student':'students number'}) \
    .sort_values('students number', ascending = False)

,RFM Score,mean_score,mean_time,finalizability,students number
6,212,66.42,240.95,100.00,693
5,211,86.69,241.29,100.00,586
10,311,88.27,232.26,100.00,546
11,312,66.87,232.13,100.00,541
7,213,48.97,241.01,100.00,473
12,313,48.20,231.51,100.00,408
0,111,90.84,244.70,100.00,369
1,112,66.89,244.88,100.00,247
9,233,30.82,241.24,0.00,217
2,113,48.36,244.66,100.00,206


We see that cluster 111 is the most successful, but the biggest claster of students is 212 with high completion rate and average time of passing exams and average number of points.